In [24]:
# Dependencies
import os
import warnings
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
import openai
import yfinance as yf
from datetime import datetime, timedelta

In [25]:
# Warning control
warnings.filterwarnings('ignore')

In [26]:
# Load environment variables from the .env file
load_dotenv()

# Fetch API keys and model name from the .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME", "gpt-3.5-turbo")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Set API keys
openai.api_key = OPENAI_API_KEY

# Instantiate tools
news_tool = SerperDevTool(api_key=SERPER_API_KEY)
scrape_tool = ScrapeWebsiteTool()

In [27]:
# Agent: News Fetcher
news_fetcher = Agent(
    role="News Fetcher",
    goal="Fetch the latest news articles about {company_name}",
    backstory=(
        "You collect the most recent news articles related to {company_name} "
        "from various reputable sources for sentiment analysis."
    ),
    allow_delegation=False,
    verbose=True,
    tools=[news_tool, scrape_tool]
)

In [28]:
# Agent: Sentiment Analyzer
sentiment_analyzer = Agent(
    role="Sentiment Analyzer",
    goal="Analyze the sentiment of recent news articles about {company_name}",
    backstory=(
        "You analyze the sentiment of the news articles, scoring each one between "
        "-1 (very negative) and 1 (very positive), and provide an aggregate sentiment score."
    ),
    allow_delegation=False,
    verbose=True
)

In [29]:
# Agent: Report Generator
report_generator = Agent(
    role="Report Generator",
    goal="Compile the sentiment scores and generate a summary report for {company_name}",
    backstory=(
        "You summarize the sentiment analysis results into an easy-to-understand report "
        "for stakeholders, highlighting key insights and trends."
    ),
    allow_delegation=False,
    verbose=True
)

In [30]:
# Task: Fetch News Articles
fetch_news = Task(
    description=(
        "1. Collect recent news articles about {company_name} from reliable sources.\n"
        "2. For each article, provide the title, publication date, content, and URL.\n"
        "3. Ensure the articles are recent (from the past week)."
    ),
    expected_output="A list of recent news articles with titles, dates, content, and URLs.",
    agent=news_fetcher,
    code="""
def run(self, inputs):
    company_name = inputs['company_name']
    query = f"{company_name} stock news"
    search_results = self.tools[0].run(query)
    articles = []
    for result in search_results['organic']:
        article = {
            'title': result.get('title'),
            'publishedAt': result.get('date'),
            'content': result.get('snippet'),
            'url': result.get('link')
        }
        if article['publishedAt']:
            articles.append(article)
    return {'articles': articles}
"""
)

In [31]:
# Task: Analyze Sentiment
analyze_sentiment = Task(
    description=(
        "1. For each article, analyze the sentiment towards {company_name}.\n"
        "2. Score the sentiment between -1 (very negative) and 1 (very positive).\n"
        "3. Provide a brief explanation for each sentiment score."
    ),
    expected_output="A list of articles with sentiment scores and explanations.",
    agent=sentiment_analyzer,
    code="""
def run(self, inputs):
    import openai
    openai.api_key = os.getenv("OPENAI_API_KEY")
    articles = inputs['articles']
    analyzed_articles = []
    for article in articles:
        prompt = (
            f"Analyze the following news article for sentiment towards {inputs['company_name']}:\n\n"
            f"Title: {article['title']}\n\n"
            f"{article['content']}\n\n"
            "Provide a sentiment score between -1 (very negative) and 1 (very positive), "
            "and a brief explanation for the score."
        )
        response = openai.ChatCompletion.create(
            model=os.getenv("OPENAI_MODEL_NAME", "gpt-3.5-turbo"),
            messages=[{"role": "user", "content": prompt}]
        )
        sentiment_analysis = response['choices'][0]['message']['content']
        analyzed_articles.append({
            'title': article['title'],
            'publishedAt': article['publishedAt'],
            'content': article['content'],
            'sentiment_analysis': sentiment_analysis.strip(),
            'url': article['url']
        })
    return {'analyzed_articles': analyzed_articles}
"""
)

In [32]:
# Task: Generate Report
generate_report = Task(
    description=(
        "1. Compile the sentiment analysis results into a summary report for {company_name}.\n"
        "2. Include the overall sentiment trend (e.g., mostly positive, negative, or neutral).\n"
        "3. Highlight key articles that significantly influence the sentiment.\n"
        "4. Provide any notable patterns or insights."
    ),
    expected_output="A summary report including overall sentiment, key articles, and trends.",
    agent=report_generator,
    code="""
def run(self, inputs):
    analyzed_articles = inputs['analyzed_articles']
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    for article in analyzed_articles:
        if 'score' in article:
            score = article['score']
        else:
            # Extract score from sentiment_analysis text
            import re
            match = re.search(r"Score:\s*([-+]?\d*\.\d+|\d+)", article['sentiment_analysis'])
            if match:
                score = float(match.group(1))
            else:
                score = 0  # Default to neutral if no score found
            article['score'] = score
        if score > 0.1:
            positive_count += 1
        elif score < -0.1:
            negative_count += 1
        else:
            neutral_count += 1
    total = positive_count + negative_count + neutral_count
    if positive_count > negative_count and positive_count > neutral_count:
        overall_sentiment = "Positive"
    elif negative_count > positive_count and negative_count > neutral_count:
        overall_sentiment = "Negative"
    else:
        overall_sentiment = "Neutral"
    report = f"Sentiment Analysis Report for {inputs['company_name']}\n"
    report += f"Date Range: Last 7 days\n"
    report += f"Total articles analyzed: {total}\n"
    report += f"Positive articles: {positive_count}\n"
    report += f"Negative articles: {negative_count}\n"
    report += f"Neutral articles: {neutral_count}\n"
    report += f"Overall Sentiment: {overall_sentiment}\n\n"
    report += "Key Articles Influencing Sentiment:\n"
    # Sort articles based on their scores
    sorted_articles = sorted(analyzed_articles, key=lambda x: abs(x['score']), reverse=True)
    for article in sorted_articles[:5]:  # Show the top 5 articles by sentiment magnitude
        report += f"- Title: {article['title']}\n"
        report += f"  Published At: {article['publishedAt']}\n"
        report += f"  Sentiment Score: {article['score']}\n"
        report += f"  Explanation: {article['sentiment_analysis']}\n"
        report += f"  URL: {article['url']}\n\n"
    return {'report': report}
"""
)

In [33]:
# Define the Crew and assign Tasks
financial_analysis_crew = Crew(
    agents=[news_fetcher, sentiment_analyzer, report_generator],
    tasks=[fetch_news, analyze_sentiment, generate_report],
    verbose=True
)


2024-10-24 16:59:37,909 - 140704492328576 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [34]:
# Specify the company
company_name = "Apple"

In [35]:
# Start the process
result = financial_analysis_crew.kickoff(inputs={"company_name": company_name})


# Agent: News Fetcher
## Task: 1. Collect recent news articles about Apple from reliable sources.
2. For each article, provide the title, publication date, content, and URL.
3. Ensure the articles are recent (from the past week).


# Agent: News Fetcher
## Thought: I need to gather recent news articles about Apple from reliable sources. I'll start by searching the internet for the latest articles published in the past week.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Apple news articles past week\"}"
## Tool Output: 

Search results: Title: Apple News+ - Apple
Link: https://www.apple.com/apple-news/
Snippet: Get the latest news from local, national, and international titles. Solve daily puzzles created exclusively for Apple News+. Listen to the week's best articles ...
---
Title: Newsroom - Apple
Link: https://www.apple.com/newsroom/
Snippet: Apple Newsroom is the source for news about Apple. Read press releases, get updates, watch video and download images

In [36]:
# Print the report
print(result.outputs['gererate_report']['report'])

AttributeError: 'CrewOutput' object has no attribute 'output'